# Reactor Kinetics Example 

Jialu Wang (jwang44@nd.edu) and Alex Dowling (adowling@nd.edu)

University of Notre Dame

This notebook conducts design of experiments for a reactor kinetics experiment with the Pyomo.DOE.
    

## Step 0: Import Pyomo and Pyomo.DOE module

In [1]:
import matplotlib.pyplot as plt
from pyomo.environ import *
from pyomo.dae import *

import numpy as np
#from scipy.interpolate import interp2d
import pandas as pd 
#from itertools import permutations, product, combinations
import idaes

from fim_doe import *

## Step 1: Import Reaction Example Mathematical Model

Consider two chemical reactions that converts molecule $A$ to desired product $B$ and a less valuable side-product $C$.

$A \overset{k_1}{\rightarrow} B \overset{k_2}{\rightarrow} C$

Our ultimate goals is to design a large-scale continous reactor that maximizes the production of $B$. This general sequential reactions problem is widely applicable to CO$_2$ capture and industry more broadly (petrochemicals, pharmasuticals, etc.).

The rate laws for these two chemical reactions are:

$r_A = -k_1 C_A$

$r_B = k_1 C_A - k_2 C_B$

$r_C = k_2 C_B$

Here, $C_A$, $C_B$, and $C_C$ are the concentrations of each species. The rate constants $k_1$ and $k_2$ depend on temperature as follows:

$k_1 = A_1 \exp{\frac{-E_1}{R T}}$

$k_2 = A_2 \exp{\frac{-E_2}{R T}}$

$A_1, A_2, E_1$, and $E_2$ are fitted model parameters. $R$ is the ideal-gas constant and $T$ is absolute temperature.

Using the **CCSI$^2$ toolset**, we would like do the following perform:

Perform **uncertainty quantification** and **design of experiments** on a small-scale **batch reactor** to infer parameters $A_1$, $A_2$, $E_1$, and $E_2$.

### Batch reactor

The concenrations in a batch reactor evolve with time per the following differential equations:

$$ \frac{d C_A}{dt} = r_A = -k_1 C_A $$

$$ \frac{d C_B}{dt} = r_B = k_1 C_A - k_2 C_B $$

$$ \frac{d C_C}{dt} = r_C = k_2 C_B $$

This is a linear system of differential equations. Assuming the feed is only species $A$, i.e., 

$$C_A(t=0) = C_{A0} \quad C_B(t=0) = 0 \quad C_C(t=0) = 0$$

When the temperature is constant, it leads to the following analytic solution:

$$C_A(t) = C_{A,0} \exp(-k_1 t)$$

$$C_B(t) = \frac{k_1}{k_2 - k_1} C_{A,0} \left[\exp(-k_1 t) - \exp(-k_2 t) \right]$$

$$C_C(t) = C_{A,0} - \frac{k_2}{k_2 - k_1} C_{A,0} \exp(-k_1 t) + \frac{k_1}{k_2 - k_1} \exp(-k_2 t) C_{A,0} = C_{A,0} - C_{A}(t) - C_{B}(t)$$

In [2]:
from reactor_kinetics import *

## Step 2: Define inputs

In [3]:
# Create model function
createmod = create_model

# discretization by Pyomo.DAE
disc = disc_for_measure

# Control time set [h]
t_control = [0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1]
    
# Measurement time points [h]
t_measure = [0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1]

# design variable and its control time set
dv_pass = {'CA0': [0],'T': t_control}
    
# Create measurement object
measure_pass = {'C':{'CA': t_measure, 'CB': t_measure, 'CC': t_measure}}
measure_class =  Measurements(measure_pass)

# Define parameter nominal value 
parameter_dict = {'A1': 84.79085853498033, 'A2': 371.71773413976416, 'E1': 7.777032028026428, 'E2': 15.047135137500822}

All measurements are flattened.
Flatten measurement name: ['C_index_CA', 'C_index_CB', 'C_index_CC']


In [4]:
def generate_exp(t_set, CA0, T):  
    '''Generate experiments. 
    t_set: time control set for T.
    CA0: CA0 value
    T: A list of T 
    '''
    assert(len(t_set)==len(T)), 'T should have the same length as t_set'
    
    T_con_initial = {}
    for t, tim in enumerate(t_set):
        T_con_initial[tim] = T[t]
        
    dv_dict_overall = {'CA0': {0: CA0},'T': T_con_initial}
    return dv_dict_overall

In [5]:
# empty prior
prior_all = np.zeros((4,4))


# add prior information
#prior_5_300 = pd.read_csv('fim_5_300_scale.csv')
#prior_5_300_500 = pd.read_csv('fim_5_300_500_scale.csv')

#prior_all = prior_5_300


prior_pass=np.asarray(prior_all)

#L_initials = np.linalg.cholesky(prior_pass)
#print(L_initials)

print('The prior information FIM:', prior_pass)
print('Prior Det:', np.linalg.det(prior_pass))
print('Eigenvalue of the prior experiments FIM:', np.linalg.eigvals(prior_pass))
print('Eigenvalue of the prior experiments FIM:', np.linalg.eigh(prior_pass)[1])

The prior information FIM: [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
Prior Det: 0.0
Eigenvalue of the prior experiments FIM: [0. 0. 0. 0.]
Eigenvalue of the prior experiments FIM: [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


## Method: Compute FIM 

This method computes an MBDoE optimization problem with no Degree of Freedom.

In [6]:
# choose from 'sequential_finite', 'direct_kaug'
# 'sequential_sipopt', 'sequential_kaug' is also available
sensi_opt = 'sequential_finite'
#sensi_opt = 'direct_kaug'

# model option
if sensi_opt == 'direct_kaug':
    args_ = [False]
else:
    args_ = [True]
    

# Define experiments
exp1 = generate_exp(t_control, 5, [570, 300, 300, 300, 300, 300, 300, 300, 300])

In [7]:
doe_object = DesignOfExperiments(parameter_dict, dv_pass,
                                 measure_class, createmod,
                                prior_FIM=prior_pass, discretize_model=disc, args=args_)


result = doe_object.compute_FIM(exp1,mode=sensi_opt, FIM_store_name = 'dynamic.csv', 
                                store_output = 'store_output', read_output=None,
                                formula='central')


result.calculate_FIM(doe_object.design_values)


This scenario: {'A1': {0: 84.8756493935153}, 'A2': {0: 371.71773413976416}, 'E1': {0: 7.777032028026428}, 'E2': {0: 15.047135137500822}, 'jac-index': {'A1': [0], 'A2': [0], 'E1': [0], 'E2': [0]}, 'eps-abs': {'A1': 0.0848756493935153, 'A2': 0.37171773413976417, 'E1': 0.007777032028026428, 'E2': 0.015047135137500823}, 'scena-name': [0]}
Ipopt 3.13.2: linear_solver=ma57
halt_on_ampl_error=yes
max_iter=3000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/

Output this time:  [5.0, 0.6706243700020834, 0.4195560837359151, 0.2624827180069096, 0.16421446362737377, 0.10273586874285646, 0.06427362421801934, 0.04021087104893356, 0.02515672906555432, 0.0, 1.4356608195123035, 1.5205901724705073, 1.5079974275825079, 1.4413335440984965, 1.347049388125238, 1.241037622436591, 1.1326547091791959, 1.027229845637414, 0.0, 2.8937148104856125, 3.0598537437935778, 3.229519854410583, 3.39445199227413, 3.5502147431319058, 3.69468875334539, 3.8271344197718706, 3.947613425297032]
This scenario: {'A1': {0: 84.79085853498033}, 'A2': {0: 371.71773413976416}, 'E1': {0: 7.784809060054453}, 'E2': {0: 15.047135137500822}, 'jac-index': {'A1': [0], 'A2': [0], 'E1': [0], 'E2': [0]}, 'eps-abs': {'A1': 0.08479085853498033, 'A2': 0.37171773413976417, 'E1': 0.007784809060054453, 'E2': 0.015047135137500823}, 'scena-name': [0]}
Ipopt 3.13.2: linear_solver=ma57
halt_on_ampl_error=yes
max_iter=3000


******************************************************************************

Output this time:  [5.0, 0.6706243700020836, 0.4195560837359153, 0.2624827180069097, 0.16421446362737382, 0.10273586874285653, 0.06427362421801941, 0.04021087104893361, 0.02515672906555436, 0.0, 1.441416665040551, 1.5268448073993228, 1.514721262165308, 1.4484455916701346, 1.3544479171661528, 1.248617623361608, 1.1403173456620836, 1.03488709459508, 0.0, 2.887958964957366, 3.0535991088647614, 3.222796019827782, 3.3873399447024912, 3.542816214090991, 3.687108752420372, 3.819471783288983, 3.939956176339366]
This scenario: {'A1': {0: 84.70606767644534}, 'A2': {0: 371.71773413976416}, 'E1': {0: 7.777032028026428}, 'E2': {0: 15.047135137500822}, 'jac-index': {'A1': [0], 'A2': [0], 'E1': [0], 'E2': [0]}, 'eps-abs': {'A1': 0.08470606767644534, 'A2': 0.37171773413976417, 'E1': 0.007777032028026428, 'E2': 0.015047135137500823}, 'scena-name': [0]}
Ipopt 3.13.2: linear_solver=ma57
halt_on_ampl_error=yes
max_iter=3000


******************************************************************************
T

Output this time:  [5.0, 0.6706243700020837, 0.4195560837359155, 0.26248271800690987, 0.16421446362737402, 0.10273586874285656, 0.06427362421801945, 0.04021087104893362, 0.02515672906555435, 0.0, 1.4384112269396816, 1.5233649787335601, 1.5108005401811877, 1.4441529235617967, 1.3498659292064588, 1.243830257365472, 1.135403210540418, 1.029916244982974, 0.0, 2.890964403058235, 3.0570789375305245, 3.2267167418119023, 3.3916326128108296, 3.547398202050685, 3.691896118416508, 3.8243859184106483, 3.9449270259514715]
This scenario: {'A1': {0: 84.79085853498033}, 'A2': {0: 371.71773413976416}, 'E1': {0: 7.769254995998401}, 'E2': {0: 15.047135137500822}, 'jac-index': {'A1': [0], 'A2': [0], 'E1': [0], 'E2': [0]}, 'eps-abs': {'A1': 0.08479085853498033, 'A2': 0.37171773413976417, 'E1': 0.0077692549959984016, 'E2': 0.015047135137500823}, 'scena-name': [0]}
Ipopt 3.13.2: linear_solver=ma57
halt_on_ampl_error=yes
max_iter=3000


*************************************************************************

Output this time:  [5.0, 0.670624370002084, 0.41955608373591563, 0.2624827180069099, 0.16421446362737396, 0.10273586874285658, 0.06427362421801945, 0.040210871048933616, 0.02515672906555434, 0.0, 1.4326573615440616, 1.5171119900116485, 1.5040785376836843, 1.4370435698520607, 1.342471604700488, 1.2362565062752762, 1.1277492573498347, 1.0222703526130221, 0.0, 2.8967182684538546, 3.0633319262524363, 3.2334387443094053, 3.398741966520566, 3.554792526556655, 3.699469869506704, 3.8320398716012316, 3.9525729183214233]
Build time with sequential_finite mode [s]: 0.033286094665527344
Solve time with sequential_finite mode [s]: 1.1756055355072021
Total wall clock time [s]: 1.8546075820922852
Existed information has been added.
FIM: [[ 2.45738410e-03  5.94863782e-04 -4.56435589e-02 -6.00876165e-02]
 [ 5.94863782e-04  2.22810780e-04 -1.03935357e-02 -2.23273539e-02]
 [-4.56435589e-02 -1.03935357e-02  8.61211473e-01  1.05437230e+00]
 [-6.00876165e-02 -2.23273539e-02  1.05437230e+00  2.27036487e+00]]

In [8]:
mod_eg = doe_object.models[0]

t_list = []
for t in mod_eg.t:
    t_list.append(t)
    
print('kp1:')
for i in t_list:
    print(value(mod_eg.kp1[0,i]))
    
print('kp2:')
for i in t_list:
    print(value(mod_eg.kp2[0,i]))
    
print('CA:')
for i in t_list:
    print(value(mod_eg.C[0,'CA',i]))
    
print('CB:')
for i in t_list:
    print(value(mod_eg.C[0,'CB',i]))
    
print('CC:')
for i in t_list:
    print(value(mod_eg.C[0,'CC',i]))

kp1:
16.447958186505904
16.447958186505904
16.447958186505904
16.447958186505904
16.447958186505904
16.447958186505904
16.447958186505904
16.447958186505904
16.447958186505904
16.447958186505904
16.447958186505904
16.447958186505904
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.7558478281198764
3.75584

In [9]:
print('======Result summary======')
print('Four design criteria log10() value:')
print('A-optimality:', np.log10(result.trace))
print('D-optimality:', np.log10(result.det))
print('E-optimality:', np.log10(result.min_eig))
print('Modified E-optimality:', np.log10(result.cond))

======Result summary======
Four design criteria log10() value:
A-optimality: 0.49613453997436285
D-optimality: -10.3118203437022
E-optimality: -5.553641798357302
Modified E-optimality: 6.006360465080967


## Method: Optimization
Gradient-based optimization with IPOPT with .optimize_doe()

This function solves twice: It solves the square version of the MBDoE problem firstly, and then unfixes the design variables as Degree of Freedoms and solves again. In this way the optimization problem can be well initialized. 

In [18]:
exp1 = generate_exp(t_control, 5, [477.43, 300, 300, 300, 300, 300, 300, 300, 300])
#exp1 = generate_exp(t_control, 5, [500, 500, 500, 500, 500, 500, 500, 500, 500])

In [ ]:
from itertools import permutations, product
doe_object = DesignOfExperiments(parameter_dict, dv_pass,
                                 measure_class, createmod,
                                prior_FIM=prior_pass, discretize_model=disc, args=[True])

square_result, optimize_result= doe_object.optimize_doe(exp1, if_optimize=True, if_Cholesky=True, 
                                                         scale_nominal_param_value=True, objective_option='det', 
                                                         L_initial=None)

#square_result = doe_object.optimize_doe(exp1, if_optimize=False, if_Cholesky=True, 
#                                                         scale_nominal_param_value=True, objective_option='det', 
#                                                         L_initial=None)

Sensitivity information is scaled by its corresponding parameter nominal value.
    'pyomo.core.base.objective.ScalarObjective'>) on block unknown with a new
    Component (type=<class 'pyomo.core.base.objective.ScalarObjective'>). This
    is usually indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
Ipopt 3.13.2: linear_solver=ma57
halt_on_ampl_error=yes
max_iter=3000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10  1.3427454e+01 8.42e+05 6.42e+06  -1.0 2.07e+04   0.5 3.74e-06 7.75e-04h  1
Reallocating memory for MA57: lfact (513180)
Reallocating memory for MA57: lfact (962390)
Reallocating memory for MA57: lfact (1058577)
  11  1.3743846e+01 8.41e+05 9.90e+06  -1.0 2.76e+04  -0.0 3.96e-04 4.49e-04h  1
Reallocating memory for MA57: lfact (2148001)
Reallocating memory for MA57: lfact (2480350)
  12  1.4067657e+01 8.41e+05 1.30e+07  -1.0 6.79e+04   0.4 3.60e-04 1.05e-04h  1
  13r 1.4067657e+01 8.41e+05 9.99e+02   5.2 0.00e+00   0.8 0.00e+00 3.43e-07R  4
  14r 6.2870147e+00 8.41e+05 2.22e+04   5.2 1.61e+08    -  5.15e-04 2.26e-05f  1
  15r 1.2703380e+01 8.40e+05 6.15e+04   4.5 1.10e+08    -  2.29e-01 4.80e-04f  1
  16  1.2622044e+01 8.26e+05 9.01e+02  -1.0 6.47e+05    -  2.08e-02 1.74e-02h  1
  17  1.2621332e+01 8.26e+05 9.01e+02  -1.0 3.82e+05    -  3.90e-02 2.10e-04h  1
  18  1.3775121e+01 7.35e+05 3.15e+05  -1.0

 102 -1.7730034e+01 1.41e+05 2.42e+05  -1.0 4.68e+05    -  6.35e-02 7.10e-04h 10
 103 -1.7739196e+01 1.41e+05 2.42e+05  -1.0 4.95e+05    -  2.69e-01 1.51e-04h 11
 104 -1.7751987e+01 1.41e+05 2.42e+05  -1.0 5.00e+05    -  3.38e-02 2.00e-04h 11
 105 -1.3414372e+01 9.87e+06 2.29e+05  -1.0 5.33e+05    -  1.00e+00 1.20e-01w  1
 106 -1.3503480e+01 9.59e+06 2.22e+05  -1.0 3.92e+03  -3.9 9.61e-02 2.81e-02w  1
 107 -1.3496035e+01 9.54e+06 2.21e+05  -1.0 6.91e+03  -4.3 2.04e-02 5.67e-03w  1
 108 -1.7757875e+01 1.41e+05 2.42e+05  -1.0 2.53e+03  -3.9 1.00e+00 5.88e-05h 11
 109 -1.7767475e+01 1.41e+05 2.42e+05  -1.0 4.82e+04    -  5.91e-02 1.11e-03h  5
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 110 -1.7776588e+01 1.41e+05 2.42e+05  -1.0 4.73e+05    -  4.18e-02 1.75e-04h 12
 111 -1.7784699e+01 1.41e+05 2.42e+05  -1.0 5.08e+05    -  1.70e-01 8.53e-05h 13
 112 -1.7790213e+01 1.41e+05 2.42e+05  -1.0 5.45e+05    -  1.73e-02 4.17e-05h 14
 113 -1.7803351e+01 1.41e+05

 196 -2.0412096e+01 1.22e+05 1.48e+04  -1.0 2.26e+05    -  6.07e-01 2.67e-03h  9
 197 -2.0807438e+01 1.21e+05 1.50e+04  -1.0 2.17e+05    -  7.29e-02 1.07e-02h  7
 198 -2.1185438e+01 1.20e+05 1.66e+04  -1.0 2.28e+05    -  2.41e-01 8.35e-03h  7
 199 -2.1442186e+01 1.19e+05 1.74e+04  -1.0 2.23e+05    -  1.17e-01 5.26e-03h  7
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 200 -2.1816055e+01 1.18e+05 1.77e+04  -1.0 2.22e+05    -  6.82e-02 9.24e-03h  6
 201 -7.7814443e+00 9.95e+04 1.08e+05  -1.0 2.34e+05    -  1.00e+00 1.74e-01H  1
 202 -7.7902829e+00 9.94e+04 1.08e+05  -1.0 7.51e+04  -5.9 7.08e-02 3.42e-04h 11
 203 -7.8004179e+00 9.94e+04 1.08e+05  -1.0 1.88e+05  -6.4 4.54e-02 1.35e-04h 11
 204 -7.8097401e+00 9.94e+04 1.08e+05  -1.0 8.13e+04  -6.0 3.09e-01 3.53e-04h 11
 205 -7.9787159e+00 9.93e+04 1.08e+05  -1.0 7.04e+05  -6.4 7.42e-03 6.82e-04h  7
 206 -7.9823683e+00 9.93e+04 1.08e+05  -1.0 6.76e+04  -6.0 1.28e-01 2.25e-04h 11
 207 -7.9839690e+00 9.93e+04

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 290 -1.5428469e+01 4.92e+04 1.41e+06  -1.0 9.99e+02   0.2 1.78e-02 1.41e-03h  1
 291 -1.6799823e+01 4.90e+04 1.46e+06  -1.0 1.22e+03  -0.3 1.86e-02 3.22e-03h  5
 292 -1.7419331e+01 4.88e+04 1.73e+06  -1.0 1.17e+03   0.1 1.68e-01 3.96e-03h  3
 293 -1.8057313e+01 4.88e+04 1.73e+06  -1.0 4.50e+03  -0.3 1.18e-03 6.39e-04h  4
 294 -1.8315616e+01 4.87e+04 1.78e+06  -1.0 1.49e+03   0.1 1.06e-01 1.78e-03h  6
 295 -1.8444125e+01 4.87e+04 1.78e+06  -1.0 6.99e+03  -0.4 1.53e-03 1.90e-04h  5
 296 -1.8715497e+01 4.86e+04 1.85e+06  -1.0 1.74e+03   0.0 8.21e-02 2.08e-03h  5
 297 -1.8777885e+01 4.86e+04 1.85e+06  -1.0 9.40e+03  -0.4 1.55e-03 8.93e-05h  6
 298 -1.8903843e+01 4.85e+04 1.85e+06  -1.0 2.07e+03  -0.0 1.45e-01 8.55e-04h  6
 299 -1.9084070e+01 4.85e+04 1.85e+06  -1.0 7.31e+03  -0.5 2.14e-03 2.10e-04h  7
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 300 -1.9277120e+01 4.85e+04

 382 -3.4427077e+01 9.78e+05 7.93e+07  -1.0 6.45e+02   0.7 5.07e-03 1.27e-03h  1
 383 -3.4318213e+01 9.22e+05 1.28e+08  -1.0 5.46e+02   1.1 5.28e-03 5.78e-02h  1
 384 -3.4316882e+01 9.21e+05 1.28e+08  -1.0 5.46e+02   1.6 6.31e-02 7.48e-04h  1
 385r-3.4316882e+01 9.21e+05 1.00e+03   6.0 0.00e+00   1.1 0.00e+00 3.03e-07R  6
 386r-3.4048253e+01 7.83e+05 1.71e+05   6.0 6.16e+08    -  1.44e-07 2.33e-04f  1
 387 -3.2468555e+01 2.20e+06 1.19e+03  -1.0 1.31e+05    -  3.09e-01 8.24e-01h  1
 388 -2.4156179e+01 1.64e+06 2.05e+03  -1.0 1.87e+05    -  3.44e-01 3.08e-01h  1
 389 -2.3806452e+01 1.64e+06 1.80e+04  -1.0 7.83e+02   0.6 5.58e-03 1.88e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 390 -2.3775992e+01 1.64e+06 1.80e+04  -1.0 1.15e+03   0.1 1.76e-02 1.23e-04h  1
 391 -1.4570425e+01 1.62e+06 5.93e+04  -1.0 1.24e+03  -0.3 3.12e-02 7.84e-03h  1
 392 -1.1636240e+01 1.62e+06 6.83e+04  -1.0 2.78e+03  -0.8 3.94e-02 2.96e-03h  1
 393 -1.1634160e+01 1.62e+06

 475 -1.3703604e+01 2.20e+04 2.51e+10  -1.0 1.02e+01   8.9 2.92e-02 1.20e-01h  1
 476 -1.3706313e+01 2.19e+04 2.51e+10  -1.0 8.99e+00   9.4 3.97e-01 3.77e-03h  1
 477 -1.3725616e+01 2.13e+04 2.46e+10  -1.0 8.95e+00   8.9 1.05e-01 2.71e-02h  1
 478 -1.3729447e+01 2.12e+04 2.46e+10  -1.0 8.72e+00   9.3 3.97e-01 5.60e-03h  1
 479 -1.3743308e+01 2.07e+04 2.42e+10  -1.0 8.67e+00   8.8 9.64e-03 2.05e-02h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 480 -1.3756484e+01 2.03e+04 2.40e+10  -1.0 8.49e+00   9.3 6.67e-02 2.03e-02h  1
 481 -1.3760626e+01 2.02e+04 2.39e+10  -1.0 8.33e+00   8.8 2.28e-02 6.49e-03h  1
 482 -1.3769776e+01 1.99e+04 2.38e+10  -1.0 8.27e+00   9.2 1.26e-01 1.46e-02h  1
 483 -1.3776958e+01 1.97e+04 2.35e+10  -1.0 8.15e+00   8.7 2.74e-02 1.17e-02h  1
 484 -1.3777613e+01 1.97e+04 2.35e+10  -1.0 8.06e+00   9.2 1.25e-01 1.07e-03h  1
 485 -1.3778141e+01 1.96e+04 2.35e+10  -1.0 2.45e+01   8.7 3.53e-03 8.79e-04h  1
 486 -1.3813340e+01 1.85e+04

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 570 -1.9473822e+01 4.67e+05 7.48e+02  -1.0 3.72e+06    -  3.67e-01 1.18e-02h  4
 571 -1.9654912e+01 4.63e+05 7.40e+02  -1.0 3.81e+06    -  4.83e-01 1.13e-02h  5
 572 -1.9727224e+01 4.61e+05 7.36e+02  -1.0 4.37e+06    -  2.57e-01 4.04e-03h  8
 573 -2.3579449e+01 1.73e+06 5.55e+02  -1.0 4.51e+06    -  5.48e-02 3.29e-01h  2
Reallocating memory for MA57: lfact (10358446)
Reallocating memory for MA57: lfact (17074760)
Reallocating memory for MA57: lfact (18473927)
 574 -2.3397846e+01 1.31e+06 4.48e+15  -1.0 4.29e+02  13.6 9.95e-01 2.50e-01h  3
 575 -2.3380543e+01 1.29e+06 4.49e+15  -1.0 3.51e+02  13.2 1.00e+00 1.56e-02h  7
 576 -2.1395576e+01 2.98e+05 2.07e+15  -1.0 3.50e+02  12.7 1.00e+00 1.00e+00w  1
 577 -1.7263666e+01 1.16e+05 1.32e+14  -1.0 4.55e+01  12.2 1.00e+00 1.00e+00h  1
 578 -7.4045210e+00 1.52e+05 2.66e+13  -1.0 1.53e+01  11.7 1.00e+00 7.98e-01h  1
 579 -5.2199433e+00 1.41e+05 1.30e+13  -1.0 1.76e

 662 -4.7433410e+00 3.94e+04 6.41e+12  -1.0 8.96e+03   7.8 5.00e-04 5.00e-04s 17
 663 -4.7439022e+00 3.91e+04 6.41e+12  -1.0 1.08e+04   7.3 1.20e-03 1.20e-03s 17
 664 -4.7441777e+00 3.90e+04 6.40e+12  -1.0 9.19e+03   7.7 8.46e-04 8.46e-04s 17
 665 -4.7454077e+00 3.84e+04 6.38e+12  -1.0 1.09e+04   7.3 2.57e-03 2.57e-03s 17
 666r-4.7454077e+00 3.84e+04 1.00e+03   3.0 0.00e+00   7.7 0.00e+00 0.00e+00R  1
 667r-1.8746890e+01 6.76e+04 1.61e+03   3.0 9.92e+05    -  6.03e-04 9.77e-04f  1
 668 -1.8665183e+01 6.51e+04 2.01e+03  -1.0 1.75e+06    -  4.75e-01 6.21e-02h  5
 669 -1.8587567e+01 6.26e+04 1.89e+03  -1.0 1.61e+06    -  1.04e-01 6.25e-02h  5
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 670 -1.8539867e+01 5.98e+04 1.78e+03  -1.0 1.20e+06    -  1.65e-01 6.25e-02h  5
 671 -1.8511835e+01 5.70e+04 1.67e+03  -1.0 9.35e+05    -  3.01e-01 6.25e-02h  5
 672 -1.8512608e+01 5.61e+04 1.65e+03  -1.0 5.83e+05    -  3.09e-01 1.56e-02h  7
 673 -1.7954551e+01 1.15e+06

 755 -3.4225793e+01 1.15e+05 4.87e+10  -1.0 4.45e+11    -  1.43e-07 1.43e-07s  2
 756r-3.4225793e+01 1.15e+05 1.00e+03   3.6 0.00e+00  11.7 0.00e+00 0.00e+00R  1
 757r-3.4032379e+01 1.15e+05 6.03e+03   3.6 4.10e+06    -  7.16e-05 9.92e-04f  1
 758 -3.3524068e+01 1.14e+05 8.37e+01  -1.0 1.60e+06    -  2.59e-01 3.28e-03h  1
 759 -2.9711357e+01 1.17e+05 9.04e+03  -1.0 1.41e+07    -  5.77e-02 1.30e-02h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 760 -3.0328934e+01 1.21e+05 1.30e+04  -1.0 6.96e+07    -  9.50e-03 2.90e-03f  4
 761 -3.0733440e+01 1.23e+05 1.20e+04  -1.0 8.73e+07    -  8.68e-03 1.58e-03f  6
 762 -3.1139363e+01 1.25e+05 1.02e+04  -1.0 1.05e+08    -  9.00e-03 1.45e-03f  6
 763 -3.1476147e+01 1.27e+05 8.50e+03  -1.0 1.37e+08    -  1.05e-02 1.01e-03f  6
 764 -3.1778631e+01 1.29e+05 6.90e+03  -1.0 1.74e+08    -  1.25e-02 8.07e-04f  6
 765 -3.6277496e+01 1.70e+06 5.73e+05  -1.0 9.44e+07    -  3.11e-02 4.31e-02f  1
 766 -3.6271181e+01 1.68e+06

In [ ]:
from pyomo.core.kernel.component_set import ComponentSet

def large_residuals_set(block, tol=1e-5, LOUD=True):
    """
    Method to return a ComponentSet of all Constraint components with a
    residual greater than a given threshold which appear in a model.
    Args:
        block : model to be studied
        tol : residual threshold for inclusion in ComponentSet
    Returns:
        A ComponentSet including all Constraint components with a residual
        greater than tol which appear in block
    """
    large_residuals_set = ComponentSet()
    for c in block.component_data_objects(
            ctype=Constraint, active=True, descend_into=True):
        if c.active and value(c.lower - c.body()) > tol:
            large_residuals_set.add(c)
        elif c.active and value(c.body() - c.upper) > tol:
            large_residuals_set.add(c)
    
    if LOUD:
        print("All equality constraints with residuals larger than",tol)
        for c in large_residuals_set:
            print(c)
            
    return large_residuals_set

large_residuals_set(mod_eg, tol=1.0E-5)

In [ ]:
print('======Result summary======')
print('This optimization is solved with status:', optimize_result.status)
print('The result FIM is:', optimize_result.FIM)
print('Four design criteria log10() value:')
print('A-optimality:', np.log10(optimize_result.trace))
print('D-optimality:', np.log10(optimize_result.det))
print('E-optimality:', np.log10(optimize_result.min_eig))
print('Modified E-optimality:', np.log10(optimize_result.cond))

t_list = []
for t in optimize_result.model.t:
    t_list.append(t)

T_list = []
for i in t_list:
    T_list.append(value(optimize_result.model.T[i]))

print(value(optimize_result.model.T[0]))
print(value(optimize_result.model.T[1]))
print()
    
si=16
plt.rc('axes', titlesize=si)
plt.rc('axes', labelsize=si)
plt.rc('xtick', labelsize=si)
plt.rc('ytick', labelsize=si)
plt.rc('legend', fontsize=12)
    
plt.plot(t_list, T_list, 'b', linewidth=2)
#plt.scatter(t_list, T_list, 'b')
plt.ylabel('T [$K$]')
plt.xlabel('Time [$h$]')
plt.show()

## Method: Exploratory analysis (Enumeration)

This method conducts exploratory analysis by enumeration. 
It allows to define any number (dimensions) of design variables.
Heatmaps can be drawn by two design variables, fixing other design variables; 
1D curve can be drawn by one design variable, fixing other design variables.

### Specify user inputs

In [ ]:
# Design variable ranges as lists 
design_ranges = [list(np.linspace(1,5,5)), list(np.linspace(300,700,5))]

# Design variable names 
dv_apply_name = ['CA0','T']

# Design variable should be fixed at these time points
dv_apply_time = [[0],t_control]

# Define experiments. This is a starting point of which the value does not matter
exp1 = generate_exp(t_control, 5, [300, 300, 300, 300, 300, 300, 300, 300, 300])
    
## choose from 'sequential_finite', 'direct_kaug'
#sensi_opt = 'sequential_finite'
sensi_opt = 'direct_kaug'

# model option
if sensi_opt == 'direct_kaug':
    args_ = [False]
else:
    args_ = [True]

In [ ]:
# add prior information
prior_5_300 = pd.read_csv('fim_5_300_scale.csv')
#prior_5_300_500 = pd.read_csv('fim_5_300_500_scale.csv')

prior_all = prior_5_300

prior_pass=np.asarray(prior_all)

#L_initials = np.linalg.cholesky(prior_pass)
#print(L_initials)

print('The prior information FIM:', prior_pass)
print('Prior Det:', np.linalg.det(prior_pass))

In [ ]:
doe_object = DesignOfExperiments(parameter_dict, dv_pass,
                                 measure_class, createmod,
                                prior_FIM=prior_pass, discretize_model=disc, args=args_)

all_fim = doe_object.run_grid_search(exp1, design_ranges, dv_apply_name, dv_apply_time, 
                                     mode=sensi_opt)

### 1D sensitivity curve

In [ ]:
test = all_fim.extract_criteria()

## draw 1D sensitivity curve 

fixed = {"'CA0'": 5.0}

all_fim.figure_drawing(fixed, ['T'], 'Reactor case','T [K]','$C_{A0}$ [M]' )



## Heatmap

In [ ]:
fixed = {}
all_fim.figure_drawing(fixed, ['CA0','T'], 'Reactor case','$C_{A0}$ [M]', 'T [K]' )

## Grid search for 3 design variables

In [ ]:

# Define design ranges
design_ranges = [list(np.linspace(1,5,2)),  list(np.linspace(300,700,2)), [300,500]]

# Define design variable 
# Here the two T are for different controlling time subsets
dv_apply_name = ['CA0', 'T', 'T']
dv_apply_time = [[0], [0], [0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875,1]]

# Define experiments
exp1 = generate_exp(t_control, 5, [300, 300, 300, 300, 300, 300, 300, 300, 300])

## choose from 'sequential_finite', 'direct_kaug'
#sensi_opt = 'sequential_finite'
sensi_opt = 'direct_kaug'

# model option
if sensi_opt == 'direct_kaug':
    args_ = [False]
else:
    args_ = [True]

In [ ]:
doe_object = DesignOfExperiments(parameter_dict, dv_pass,
                                 measure_class, createmod,
                                prior_FIM=prior_pass, discretize_model=disc, args=args_)

all_fim = doe_object.run_grid_search(exp1, design_ranges, dv_apply_name, dv_apply_time, 
                                     mode=sensi_opt)

### Draw 1D sensitivity curve

In [ ]:
test = all_fim.extract_criteria()

In [ ]:
## draw 1D sensitivity curve 

fixed = {"'CA0'": 1.0, "'T2'": 300}

all_fim.figure_drawing(fixed, ['T'], 'Reactor case','T [K]','$C_{A0}$ [M]' )

### Draw 2D sensitivity curve

In [ ]:
fixed = {"'T2'": 300}

all_fim.figure_drawing(fixed, ['CA0','T'], 'Reactor case','$C_{A0}$ [M]', 'T [K]' )